In [2]:
import os
from sqlitedict import SqliteDict

In [ ]:
%%time

## Creating mapping
## 1. transactionHash- Integer ID mapping

list_tx_hashes = []
path = r"C:\Users\japje\BDAProj\Outputs2015"
for root, dir, files in os.walk(path):
    for f in files:
        with open(os.path.join(root,f), encoding = "utf-8") as f:
            for line in f:
                ## Split the row
                cells = line.split("\t")

                ## Add transaciton to Set
                transaction_hash = cells[1]
                list_tx_hashes.append(transaction_hash)
                
                
list_tx_hashes = list(set(list_tx_hashes))

### Creating an Sqlitedict db
with SqliteDict('./mappings_db.sqlite', tablename='txn_mapping') as mydict:
    for i in range(len(list_tx_hashes)):
        mydict[list_tx_hashes[i]] = i
    mydict.commit()

del list_tx_hashes

In [ ]:
%%time

## Creating the mapping
## 2. Address- Integer ID mapping

list_addresses = []
path = r"C:\Users\japje\BDAProj\Outputs2015"
for root, dir, files in os.walk(path):
    for f in files:
        if("outputs" in f):
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    ## Split the row
                    cells = line.split("\t")

                    ## Add address to Set
                    addresses = cells[2:][::2]
                    list_addresses = list_addresses + addresses
                
                
list_addresses = list(set(list_addresses))
### Creating an Sqlitedict db
with SqliteDict('./mappings_db.sqlite', tablename='addr_mapping') as mydict:
    for i in range(len(list_addresses)):
        mydict[list_addresses[i]] = i
    mydict.commit()

del list_addresses
print(str(round(sys.getsizeof(addr_mapping)/(1024*1024), 2)) + " MB")

In [ ]:
%%time
## Generation of output mapping 1
## str(txnID) + "_" + str(output_index) -> OutputID
## 2. OutputID -> addressID -> amount

list_addresses = []
path = r"C:\Users\japje\BDAProj\edges2010"
mapping_tx_to_op = {}
mapping_op = {}
global_index = 0
for root, dir, files in os.walk(path):
    for f in files:
        if("outputs" in f):
            with open(os.path.join(root,f), encoding = "utf-8") as f:
                for line in f:
                    ## Split the row
                    cells = line.rstrip().split("\t")
                    # print(cells)
                    list_tx_index = list(range(0, len(cells[2:])//2))
                    list_addr_id = list(range(global_index+1, global_index+1+len(cells[2:])//2))
                    list_tx_rep = [txn_mapping[cells[1]]] * (len(cells[2:])//2)

                    list_addresses = [addr_mapping[x] for x in cells[2:][::2]]
                    list_amounts = cells[3:][::2]

                    mapping_delta1 = {x: {y: z} for x, y, z in zip(list_tx_rep, list_tx_index, list_addr_id)}

                    mapping_delta2 = {x: {y: z} for x, y, z in zip(list_addr_id, list_addresses, list_amounts)}

                    mapping_tx_to_op.update(mapping_delta1)
                    mapping_op.update(mapping_delta2)
                    global_index = global_index+1
                global_index = global_index+1


